In [29]:
# Import the dependencies.
import pandas as pd
from pandas import DataFrame
import gmaps
import requests
from config import g_key
import matplotlib.pyplot as plt


In [32]:
# get data from part1 csv
city_data_df = pd.read_csv("WeatherPy_challenge")
city_data_df.head()

,City_ID,City,Country,Weather Description,Rain inches (Last 3 hours),Snow inches (Last 3 hours),Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bermejo,AR,clear sky,0.00,0.0,2020-02-22 19:34:47,-32.88,-68.80,78.80,31,0,10.29
1,1,Plettenberg Bay,ZA,light rain,1.50,0.0,2020-02-22 19:34:47,-34.05,23.37,65.14,85,100,14.85
2,2,Port Alfred,ZA,light rain,1.63,0.0,2020-02-22 19:34:48,-33.59,26.89,66.00,96,100,3.00
3,3,Omaruru,NaN,broken clouds,0.00,0.0,2020-02-22 19:34:48,-21.43,15.93,73.04,61,52,10.11
4,4,Tonantins,BR,overcast clouds,0.00,0.0,2020-02-22 19:34:48,-2.87,-67.80,78.64,89,100,2.51


In [5]:
# Get Min and Max temp preferences
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_input = input("Do you want it to be raining? (yes/no) ")
snow_input = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
#filter dataframe to include cities within temperature preferences
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Weather Description,Rain inches (Last 3 hours),Snow inches (Last 3 hours),Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bermejo,AR,clear sky,0.00,0.0,2020-02-22 19:34:47,-32.88,-68.80,78.80,31,0,10.29
3,3,Omaruru,NaN,broken clouds,0.00,0.0,2020-02-22 19:34:48,-21.43,15.93,73.04,61,52,10.11
4,4,Tonantins,BR,overcast clouds,0.00,0.0,2020-02-22 19:34:48,-2.87,-67.80,78.64,89,100,2.51
8,8,Hilo,US,clear sky,0.00,0.0,2020-02-22 19:33:08,19.73,-155.09,75.20,64,1,6.93
9,9,Saint-Pierre,RE,overcast clouds,0.00,0.0,2020-02-22 19:30:24,-21.34,55.48,75.20,78,100,5.82
13,13,Saint-Philippe,RE,light rain,0.56,0.0,2020-02-22 19:34:50,-21.36,55.77,75.20,78,72,5.82
18,18,Sechura,PE,clear sky,0.00,0.0,2020-02-22 19:34:51,-5.56,-80.82,74.12,87,0,16.80
24,24,Punta Arenas,CL,overcast clouds,0.00,0.0,2020-02-22 19:34:53,-53.15,-70.92,75.20,38,100,23.04
34,34,Tessalit,ML,broken clouds,0.00,0.0,2020-02-22 19:34:55,20.20,1.01,75.81,9,62,12.48
46,46,Carnarvon,AU,scattered clouds,0.00,0.0,2020-02-22 19:34:58,-24.87,113.63,75.20,83,40,6.93


In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Weather Description","Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [19]:
hotel_df

,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
0,Bermejo,AR,clear sky,78.80,-32.88,-68.80,
3,Omaruru,NaN,broken clouds,73.04,-21.43,15.93,
4,Tonantins,BR,overcast clouds,78.64,-2.87,-67.80,
8,Hilo,US,clear sky,75.20,19.73,-155.09,
9,Saint-Pierre,RE,overcast clouds,75.20,-21.34,55.48,
...,...,...,...,...,...,...,...
550,Abu Zabad,SD,clear sky,70.45,12.35,29.25,
552,Atambua,ID,scattered clouds,74.75,-9.11,124.89,
554,Conselheiro Pena,BR,overcast clouds,79.66,-19.17,-41.47,
560,Windhoek,NaN,light rain,70.00,-22.56,17.08,


In [20]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [22]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
hotel_df

,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
0,Bermejo,AR,clear sky,78.80,-32.88,-68.80,Hotel Mendoza
3,Omaruru,NaN,broken clouds,73.04,-21.43,15.93,"Kashana, Omaruru, Namibia"
4,Tonantins,BR,overcast clouds,78.64,-2.87,-67.80,Hotel Rei Davi
8,Hilo,US,clear sky,75.20,19.73,-155.09,Hilo Hawaiian Hotel
9,Saint-Pierre,RE,overcast clouds,75.20,-21.34,55.48,Lindsey Hôtel
...,...,...,...,...,...,...,...
550,Abu Zabad,SD,clear sky,70.45,12.35,29.25,محي الدين نوري
552,Atambua,ID,scattered clouds,74.75,-9.11,124.89,Pondok Pesantren Al Muhajirin Atambua
554,Conselheiro Pena,BR,overcast clouds,79.66,-19.17,-41.47,Hotel Bela Adormecida
560,Windhoek,NaN,light rain,70.00,-22.56,17.08,Hotel Uhland


In [24]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))